In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import gc
# _FOLDER = "C:/Users/junny/GitRepos/DrugProfiles_2/database/"
_FOLDER = "database/"
_FOLDER_2 = "figures/"
_FOLDER_3 = "results/"

In [21]:
os.chdir('C:/Users/junny/GitRepos/DrugProfiles_2/functions')

In [22]:
from functions.fitting import*

In [24]:
from numba import njit

@njit
def CatData(arr):
    bins = np.empty(arr.shape[0])
    for idx, x in enumerate(arr):
        if (x >= 0) & (x < 0.5):
            bins[idx] = 1
        elif (x >= 0.5) & (x < 0.7):
            bins[idx] = 2
        elif (x >= 0.7) & (x < 0.9):
            bins[idx] = 3
        elif (x >= 0.9) & (x < 0.95):
            bins[idx] = 4
        elif (x >= 0.95) & (x < 0.99):
            bins[idx] = 5
        elif x >= 0.99:
            bins[idx] = 6
        else:
            bins[idx] = 0

    return bins

def BinningFunction(df, column, new_column, categorical=True):
    df[new_column] = CatData(df[column].to_numpy())
    
    if categorical:
        map_dict = {
            1: "0-0.5",
            2: "0.5-0.7",
            3: "0.7-0.9",
            4: "0.9-0.95",
            5: "0.95-0.99",
            6: ">0.99",
            0: "<0"
        }
        df[new_column]= df[new_column].map(map_dict)
    

ModuleNotFoundError: No module named 'numba'

In [ ]:
os.listdir("results/")

In [ ]:
functions = [
             "fsigmoid", 
             "sigmoid_2_param",
             "sigmoid_3_param",
             "sigmoid_4_param",
             "logistic_4_param",
            "ll4_4_param",
            "ll4R_4_param",
            "logLogist_3_param"]

#r2_columns = [fitting_function+"_r2" for fitting_function in functions]

filtration_types = ['123', '1234', 'auc_02', 'auc_04','auc_06', "auc", "no_filt"]


In [ ]:
df_099 = pd.DataFrame(index=filtration_types, columns = functions)
df_09 = pd.DataFrame(index=filtration_types, columns = functions)

In [ ]:
%%time


for filtering in filtration_types:
    print("Filtering:", filtering)
    try:
        df = pd.read_csv(_FOLDER_3+"filt_"+filtering+".csv")
        print("Data left after filtering: %d" % df.shape[0])
        del df
    except:
        df = pd.read_csv(_FOLDER+"normalised_dose_response_data.csv")
        print("No prior filtering, original shape:", df.shape)
    
    df_1 = pd.read_csv(_FOLDER_3+"fit_"+filtering+".csv")
    print("Left after fitting:", df_1.shape[0])
    
    df_0 = pd.DataFrame()
    df_0["index"] = ['>0.99', '0.95-0.99', '0.9-0.95', '0.7-0.9', '0.5-0.7', '0-0.5', "<0", "sum_>0.9"]

    for func in functions:
        BinningFunction(df_1, func +"_r2", func) 
        df = df_1[func].value_counts().reset_index()
        df.loc[7, "index"] = "sum_>0.9" 
        df.loc[7, func]= df.iloc[:3][func].sum()
        df[func] = df[func].astype("int32")
#         display(df)
        df_0 = pd.merge(df_0, df, on="index", how="left")
    display(df_0)
    df_099.loc[filtering, :]= df_0.loc[0, functions]
    df_09.loc[filtering, :]= df_0.loc[7, functions]
    print("********************************\n")

## Fitting results

In [ ]:
#Number of drug curves with fitting R2>0.99
df_099

In [ ]:
#Number of drug curves with fitting R2>0.9
df_09

In [ ]:
max_099 = int(max(df_099.max(axis=0)))
print("Fitting>0.99:", max_099, "samples")

for col in functions:
    if max_099 in df_099[col].values:
        print("Filtering:", df_099.index[np.argwhere(df_099[col].values == max_099)][0][0])
        print("Fitting function:", col)
print("")

max_09 = int(max(df_09.max(axis=0)))
print("Fitting>0.90:", max_09, "samples")

for col in functions:
    if max_09 in df_09[col].values:
        print("Filtering:", df_09.index[np.argwhere(df_09[col].values == max_09)][0][0])
        print("Fitting function:", col)

In [ ]:
dff.head()